In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

dados_categorias = pd.read_csv('./northwind/categories.csv', delimiter=';')
dados_clientes = pd.read_csv('./northwind/customers.csv', delimiter=';')
dados_pedidos = pd.read_csv('./northwind/orders.csv', delimiter=';')
dados_detalhes = pd.read_csv('./northwind/order_details.csv', delimiter=';')
dados_produtos = pd.read_csv('./northwind/products.csv', delimiter=';')

plt.style.use('ggplot')
sns.set_palette('pastel')

In [ ]:
dados_pedidos['data_pedido'] = pd.to_datetime(dados_pedidos['order_date'])
dados_completos = pd.merge(dados_detalhes, dados_pedidos, on='order_id')
dados_completos = pd.merge(dados_completos, dados_produtos, on='product_id')

vendas_mensais = dados_completos.groupby(dados_pedidos['data_pedido'].dt.to_period('M'))['quantity'].sum()
plt.figure(figsize=(10,5))
vendas_mensais.plot(kind='line', marker='o')
plt.title('Vendas Mensais Northwind')
plt.ylabel('Quantidade Vendida')
plt.xlabel('Mês')
plt.tight_layout()

In [ ]:
clientes_ativos = dados_pedidos.groupby('customer_id')['data_pedido'].max()
churn = (pd.Timestamp.now() - clientes_ativos).dt.days > 90
taxa_churn = churn.mean()

plt.figure(figsize=(6,6))
plt.pie([taxa_churn, 1-taxa_churn], labels=['Clientes Inativos', 'Clientes Ativos'], autopct='%1.1f%%')
plt.title('Taxa de Churn nos Últimos 3 Meses')

In [ ]:
ticket_medio = dados_completos.groupby('order_id')['unit_price'].sum().mean()
produtos_top = dados_completos.groupby('product_name')['quantity'].sum().nlargest(5)

print(f'Ticket Médio: R${ticket_medio:.2f}')
produtos_top.plot(kind='barh', title='Top 5 Produtos Mais Vendidos')